In [1]:
import pandas
import numpy
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#import data #python-engine to avoid probles when importing
movies=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1") 
ratings=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/ratings.dat", sep="::", header=None, engine="python", encoding="latin-1") 
users=pandas.read_csv("Datasets/Boltzmann_Machines/ml-1m/users.dat", sep="::", header=None, engine="python", encoding="latin-1") 

In [3]:
training_set=pandas.read_csv("Datasets/Boltzmann_Machines/ml-100k/u1.base", delimiter="\t")
training_set=numpy.array(training_set, int)

In [4]:
test_set=pandas.read_csv("Datasets/Boltzmann_Machines/ml-100k/u1.test", delimiter="\t")
test_set=numpy.array(test_set, int)

In [5]:
nb_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))

In [6]:
#converting the data unto an array with users in lines and movies in columns
def convert(data):
    new_data=[]
    for id_user in range(1, nb_users+1):
        id_movies=data[:,1][data[:,0]==id_user]
        id_ratings=data[:,2][data[:,0]==id_user]
        ratings=numpy.zeros(nb_movies)
        ratings[id_movies-1]=id_ratings
        new_data.append(list(ratings))
    return new_data

In [7]:
training_set=convert(training_set)
test_set=convert(test_set)

In [8]:
training_set=torch.FloatTensor(training_set) #expect a list of lists
test_set=torch.FloatTensor(test_set)

In [9]:
#creating the architecture of the neural network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1=nn.Linear(nb_movies, 20) #20 neurons encoding
        self.fc2=nn.Linear(20, 10) #10 neurons
        self.fc3=nn.Linear(10, 20) #decoding
        self.fc4=nn.Linear(20, nb_movies)
        self.activation=nn.Sigmoid()

    def forward(self, x):
        x=self.activation(self.fc1(x))
        x=self.activation(self.fc2(x))
        x=self.activation(self.fc3(x))
        x=self.fc4(x)
        return x

In [12]:
sae=SAE()
criterion=nn.MSELoss()
optimezer=optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [28]:
#training
nb_epoch=5
for epoch in range(1, nb_epoch+1):
    train_loss=0
    s=0.
    for id_user in range(nb_users):
        Input=Variable(training_set[id_user]).unsqueeze(0)
        target=Input.clone()
        if torch.sum(target.data>0)>0:
            Output=sae.forward(Input)
            target.require_grad=False #this won't allow compute gradient with the target
            Output[target==0]=0 #this values don't count in the error calculation
            loss=criterion(Output, target)
            mean_corrector=nb_movies/float(torch.sum(target.data>0)+ 1e-10) #because we are using movies with ratings>0, not all movies
            loss.backward() #decides the direccion 
            train_loss+=numpy.sqrt(loss.data*mean_corrector)
            s+=1.
            optimezer.step() #decides the magnitud
    print("epoch:  "+str(epoch) + "  loss:  "+str(train_loss/s))

epoch:  1  loss:  tensor(0.8741)
epoch:  2  loss:  tensor(0.8722)
epoch:  3  loss:  tensor(0.8724)
epoch:  4  loss:  tensor(0.8719)
epoch:  5  loss:  tensor(0.8712)


In [47]:
test_loss=0
s=0.
for id_user in range(nb_users):
    Input=Variable(training_set[id_user]).unsqueeze(0)
    target=Variable(test_set[id_user])
    if torch.sum(target.data>0)>0:
        Output=sae.forward(Input)[0]
        target.require_grad=False #this won't allow compute gradient with the target
        Output[target==0]=0 #this values don't count in the error calculation
        loss=criterion(Output, target)
        mean_corrector=nb_movies/float(torch.sum(target.data>0)+ 1e-10) #because we are using movies with ratings>0, not all movies
        test_loss+=numpy.sqrt(loss.data*mean_corrector)
        s+=1.
print("  loss:  "+str(test_loss/s))

  loss:  tensor(0.9451)
